In [73]:
import ase
import numpy as np
from numpy.linalg import norm, svd
from ase.visualize import view
import struct_statistics as stats
import matplotlib.pyplot as p
import soaputils as su
#%matplotlib

In [2]:
struct_ran = su.gen_struct(70, seed=50)
struct_opt = ase.io.read('res_structs/x0_test/batch0.cfg')

In [ ]:
import soaplite
import genBasis
rCut = 10.0
NradBas = 5
Lmax = 5
myAlphas, myBetas = genBasis.getBasisFunc(rCut, NradBas)

at_ran = soaplite.get_periodic_soap_structure(struct_ran, myAlphas, myBetas, rCut, NradBas, Lmax)
mat_opt = soaplite.get_periodic_soap_structure(struct_opt, myAlphas, myBetas, rCut, NradBas, Lmax)
np.save("res_data/mat_ran",mat_ran)
np.save("res_data/mat_opt",mat_opt)

In [3]:
mat_ran = np.load("res_data/mat_ran.npy")
mat_opt = np.load("res_data/mat_opt.npy")

In [16]:
def display_unit_cell(struct):
    unit_cell = struct.get_cell() 
    positions_in_A = struct.get_positions()
    types = struct.get_chemical_symbols()
    Z = struct.get_atomic_numbers()
    Z[1] = 0
    # display the atom positions:
    p.scatter(positions_in_A[:,0],positions_in_A[:,1],5*Z,'b')
    
    # display the unit cell boundary:
    for j1 in range(3):
        for j2 in range(3):
            if j1 != j2:
                for j3 in range(3):
                    if (j2 != j3) and (j1 != j3):
                        x = [0, unit_cell[j1,0]]+unit_cell[j2,0]+unit_cell[j3,0]
                        y = [0, unit_cell[j1,1]]+unit_cell[j2,1]+unit_cell[j3,1]                        
                    else:
                        x = [0, unit_cell[j1,0]]+unit_cell[j2,0]
                        y = [0, unit_cell[j1,1]]+unit_cell[j2,1]
                    p.plot(x,y,'k')
            else:
                x = [0, unit_cell[j1,0]]
                y = [0, unit_cell[j1,1]]            
                p.plot(x,y,'k')
    p.axis('equal');

In [17]:
# show random structure
%matplotlib qt5
structure = struct_ran.copy()
Natom = structure.get_number_of_atoms()
unit_cell_volume =structure.get_volume()
unit_cell = structure.get_cell() 
xRange = unit_cell[0,0]
yRange = unit_cell[1,1]
print(Natom," atoms in a super cell volume of ",unit_cell_volume," A^3, area in (001) projection:",xRange,"x",yRange,"A^2\n")

# rotate the cube to make it look 3D:
structure.rotate(45,(1,1,1),rotate_cell=True)
p.subplot(1,2,1)
display_unit_cell(structure)
p.title('rotated unit cell')

# rotate back:
structure.rotate(-45,(1,1,1),rotate_cell=True)
p.subplot(1,2,2)
display_unit_cell(structure)
p.title('aligned unit cell, (001) projection')
p.show()

70  atoms in a super cell volume of  2333.3333333333344  A^3, area in (001) projection: 13.263524026321305 x 13.263524026321305 A^2



In [11]:
# show optimised structure
structure = struct_opt.copy()
Natom = structure.get_number_of_atoms()
unit_cell_volume =structure.get_volume()
unit_cell = structure.get_cell() 
xRange = unit_cell[0,0]
yRange = unit_cell[1,1]
print(Natom," atoms in a super cell volume of ",unit_cell_volume," A^3, area in (001) projection:",xRange,"x",yRange,"A^2\n")

# rotate the cube to make it look 3D:
structure.rotate(45,(1,1,1),rotate_cell=True)
p.subplot(1,2,1)
display_unit_cell(structure)
p.title('rotated unit cell')

# rotate back:
structure.rotate(-45,(1,1,1),rotate_cell=True)
p.subplot(1,2,2)
display_unit_cell(structure)
p.title('aligned unit cell, (001) projection')
p.show()

70  atoms in a super cell volume of  2333.3333194419156  A^3, area in (001) projection: 13.263524 x 13.263524 A^2



In [36]:
# show matrices
fig, axs = p.subplots(1, 2)
p.subplot(1,2,1)
p.imshow(mat_ran)

p.subplot(1,2,2)
p.imshow(mat_opt)

#p.colorbar()
p.colorbar(orientation='horizontal')
p.show()

In [72]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = p.subplots(nrows=1, ncols=2)
for ax in axes.flat:
    im = ax.imshow(mat_ran)

fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

p.colorbar(im, cax=cax)
p.show()

In [81]:
# show SVD plots
s_ran = svd(mat_ran.transpose(), full_matrices=False, compute_uv=False)
s_opt = svd(mat_opt.transpose(), full_matrices=False, compute_uv=False)

p.semilogy(s_ran)
p.semilogy(s_opt)
p.xlabel('$i$')
p.ylabel('Singular value $\sigma_i$')
p.legend(['random structure','optimised structure'])

